In [1]:
# A bit of setup

import numpy as np
import matplotlib.pyplot as plt

from __future__ import print_function

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [8]:
from cs231_utils.data_utils import load_tiny_imagenet

def get_subset_tiny_imagenet(num_training=1600, num_validation=200, num_test=200):
    """
    Load the a subset of the tiny imagenet dataset from disk and perform preprocessing to prepare
    it for our neural network classifiers. Also sets a seed to make sure we always get the same
    data sets.
    """
    # Set the seed.
    np.random.seed(231)
    
    # Load the raw tiny imagenet data. Note this by default loads normalized image data based on
    # then entire original sample.
    tiny_imagenet_dir = 'datasets/tiny-imagenet-200'
    data_dict = load_tiny_imagenet(tiny_imagenet_dir)
    X_train, y_train, X_test, y_test = (
        data_dict['X_train'], data_dict['y_train'],
        data_dict['X_val'], data_dict['y_val'],
        data_dict['X_test'], data_dict['y_test'])
        
    # Subsample training data.
    assert(num_training <= X_train.shape[0])
    print("Taking %s of %s training samples" % (num_training, X_train.shape[0]))
    mask = np.random.choice(X_train.shape[0], num_training, replace=False)
    X_train = X_train[mask]
    y_train = y_train[mask]
    assert(num_validation <= X_val.shape[0])
    print("Taking %s of %s validation samples" % (num_validation, X_val.shape[0]))
    mask = np.random.choice(X_val.shape[0], num_validation, replace=False)
    X_val = X_val[mask]
    y_val = y_val[mask]
    assert(num_test <= X_test.shape[0])
    print("Taking %s of %s test samples" % (num_test, X_test.shape[0]))
    mask = np.random.choice(X_test.shape[0], num_test, replace=False)
    X_test = X_test[mask]
    y_test = y_test[mask]

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test = get_subset_tiny_imagenet()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

loading training data for synset 20 / 200
loading training data for synset 40 / 200
loading training data for synset 60 / 200
loading training data for synset 80 / 200
loading training data for synset 100 / 200
loading training data for synset 120 / 200
loading training data for synset 140 / 200
loading training data for synset 160 / 200
loading training data for synset 180 / 200
loading training data for synset 200 / 200


ValueError: too many values to unpack (expected 4)